# ChromaDB Vector Store 구축

청크들을 임베딩하여 ChromaDB에 저장합니다.

In [ ]:
import sys
from pathlib import Path
import os
from getpass import getpass

# 프로젝트 루트 경로
project_root = Path().absolute().parent
sys.path.append(str(project_root))

from src import config
from src.vectorstore import ChromaDBManager

print(f"ChromaDB 저장 경로: {config.CHROMA_DB_DIR}")
print(f"컬렉션 이름: {config.COLLECTION_NAME}")

## 1. OpenAI API Key 설정

In [ ]:
# .env 파일에서 자동 로드 시도
api_key = config.OPENAI_API_KEY

# .env 파일에 없으면 직접 입력
if not api_key:
    print("OpenAI API Key를 입력하세요:")
    api_key = getpass("API Key: ")
    os.environ['OPENAI_API_KEY'] = api_key

print("✅ API Key 설정 완료")

## 2. 청크 데이터 로드

In [ ]:
# 이전 노트북에서 저장한 청크 불러오기
try:
    %store -r all_chunks
    print(f"✅ {len(all_chunks)}개 청크 로드 완료")
except:
    print("❌ 이전 노트북에서 저장한 청크가 없습니다.")
    print("   03_chunking_test.ipynb를 먼저 실행해주세요.")
    all_chunks = []

## 3. ChromaDB Manager 초기화

In [ ]:
# ChromaDB Manager 생성
chroma_manager = ChromaDBManager(api_key=api_key)

print("✅ ChromaDB Manager 초기화 완료")

## 4. Vector Store 생성

⚠️ 이 단계는 시간이 오래 걸릴 수 있습니다 (100개 문서 기준 5-10분)

⚠️ OpenAI API 비용이 발생합니다

In [ ]:
if all_chunks:
    print(f"{len(all_chunks)}개 청크를 ChromaDB에 저장합니다...")
    print("(이 작업은 몇 분 정도 소요될 수 있습니다)\n")
    
    # Vector Store 생성 (기존 컬렉션 삭제)
    vectorstore = chroma_manager.create_vectorstore(
        chunks=all_chunks,
        reset=True  # 기존 컬렉션 삭제 후 새로 생성
    )
    
    print("\n✅ Vector Store 생성 완료!")
else:
    print("❌ 청크 데이터가 없습니다.")

## 5. Vector Store 검증

In [ ]:
# 컬렉션 정보 확인
info = chroma_manager.get_collection_info()

print("ChromaDB 컬렉션 정보:")
print(f"  이름: {info.get('name', 'N/A')}")
print(f"  문서 개수: {info.get('count', 0):,}개")
print(f"  메타데이터: {info.get('metadata', {})}")

## 6. 검색 테스트

In [ ]:
# 간단한 검색 테스트
test_query = "이러닝 시스템 요구사항"

print(f"검색 쿼리: '{test_query}'\n")

results = chroma_manager.similarity_search(
    query=test_query,
    k=3
)

print(f"검색 결과: {len(results)}개 문서\n")

for i, doc in enumerate(results, 1):
    print(f"[{i}] 파일: {doc.metadata.get('file_name', 'Unknown')}")
    print(f"    내용: {doc.page_content[:200]}...")
    print()

In [ ]:
# 점수와 함께 검색
results_with_score = chroma_manager.similarity_search_with_score(
    query=test_query,
    k=5
)

print(f"검색 결과 (점수 포함):\n")

for i, (doc, score) in enumerate(results_with_score, 1):
    print(f"[{i}] 점수: {score:.4f}")
    print(f"    파일: {doc.metadata.get('file_name', 'Unknown')}")
    print(f"    내용: {doc.page_content[:150]}...")
    print()

## 7. 메타데이터 필터링 테스트

In [ ]:
# 특정 파일 타입만 검색 (예: PDF만)
results_filtered = chroma_manager.similarity_search(
    query=test_query,
    k=3,
    filter={"file_type": "pdf"}  # PDF 파일만
)

print(f"필터링된 검색 결과 (PDF만): {len(results_filtered)}개\n")

for i, doc in enumerate(results_filtered, 1):
    print(f"[{i}] {doc.metadata.get('file_name', 'Unknown')} ({doc.metadata.get('file_type', 'Unknown')})")
    print(f"    {doc.page_content[:150]}...\n")

## 8. Vector Store 재로드 테스트

In [ ]:
# 새로운 Manager로 기존 Vector Store 로드
new_manager = ChromaDBManager(api_key=api_key)
loaded_vectorstore = new_manager.load_vectorstore()

if loaded_vectorstore:
    print("✅ Vector Store 재로드 성공")
    
    # 재로드 후 검색 테스트
    test_results = new_manager.similarity_search(test_query, k=2)
    print(f"\n재로드 후 검색 결과: {len(test_results)}개")
else:
    print("❌ Vector Store 재로드 실패")

## 9. 다양한 검색 쿼리 테스트

In [ ]:
test_queries = [
    "사업 예산은 얼마인가요?",
    "시스템 성능 요구사항",
    "보안 관련 요구사항",
    "프로젝트 일정",
    "제출 서류 목록"
]

print("다양한 쿼리 검색 테스트:\n")

for query in test_queries:
    results = chroma_manager.similarity_search(query, k=1)
    
    if results:
        print(f"Q: {query}")
        print(f"A: {results[0].page_content[:150]}...")
        print(f"   (출처: {results[0].metadata.get('file_name', 'Unknown')})")
        print()

## 10. Vector Store 저장 확인

In [ ]:
import os

# ChromaDB 디렉토리 확인
chroma_dir = config.CHROMA_DB_DIR

if chroma_dir.exists():
    files = list(chroma_dir.rglob("*"))
    total_size = sum(f.stat().st_size for f in files if f.is_file()) / (1024 * 1024)
    
    print(f"ChromaDB 저장 상태:")
    print(f"  경로: {chroma_dir}")
    print(f"  파일 수: {len([f for f in files if f.is_file()])}개")
    print(f"  총 크기: {total_size:.2f} MB")
    print(f"\n✅ Vector Store가 디스크에 영구 저장되었습니다.")
else:
    print("❌ ChromaDB 디렉토리를 찾을 수 없습니다.")

## 11. 다음 단계

In [ ]:
# VectorStore를 다음 노트북에서 사용할 수 있도록 저장
%store chroma_manager

print("✅ ChromaDB Manager 저장 완료")
print("\n다음 단계: 05_baseline_rag.ipynb")
print("   → RAG 파이프라인 구축 및 평가")

## 요약

이 노트북에서 완료한 작업:
- ✅ ChromaDB Vector Store 생성
- ✅ 청크 임베딩 및 저장
- ✅ 유사도 검색 테스트
- ✅ 메타데이터 필터링 테스트
- ✅ Vector Store 영구 저장

다음 단계에서는 이 Vector Store를 사용하여 RAG 파이프라인을 구축합니다.